In [ ]:
import numpy as np
from netCDF4 import Dataset
import datetime
import glob
import os
import matplotlib.pyplot as plt
import cartopy
from matplotlib import gridspec
from regrid import regrid
import pandas as pd
import itertools
import cartopy.crs as ccrs
import tqdm
from scipy.stats import linregress
import warnings
warnings.filterwarnings("ignore")

In [ ]:
rho_fyi = 916.7
rho_myi = 882
icetype = np.array(Dataset('/Users/carmennab/Dropbox/met_office/data/auxiliary/icetype.nc')['Ice Type'])
icetype[icetype==2] = rho_fyi
icetype[icetype==3] = rho_myi

In [ ]:
datapath = '/Users/carmennab/Dropbox/met_office/data/experiments/FOAM_grid/'
ctrl = Dataset(datapath+'rosie_UCL_CTL_NOSIT_ASSIM_r243541_dcarneir.nc')
base = Dataset(datapath+'rosie_UCL_BASELINE_SIT_ASSIM_CS2_AWI_SNFOAM_ALPHA1_r243541_dcarneir.nc')
unc = Dataset(datapath+'rosie_UCL_SIT_ASSIM_CS2_AWI_SNFOAM_ALPHA1_NEW_OBERR_r243541_dcarneir.nc')
### alpha
nine = Dataset(datapath+'rosie_UCL_SIT_ASSIM_CS2_AWI_SNFOAM_ALPHA09_r243541_dcarneir.nc')
six = Dataset(datapath+'rosie_UCL_SIT_ASSIM_CS2_AWI_SNFOAM_ALPHA06_r243541_dcarneir.nc')
### retracker
larm = Dataset(datapath+'rosie_UCL_SIT_ASSIM_CS2_BRISTOL_SNFOAM_ALPHA1_r243541_dcarneir.nc')
cpom = Dataset(datapath+'rosie_UCL_SIT_ASSIM_CS2_CPOM_SNFOAM_ALPHA1_r243541_dcarneir.nc')
### snow
smlg = Dataset(datapath+'rosie_UCL_SIT_ASSIM_CS2_AWI_SNLG_ALPHA1_r243541_dcarneir.nc')
awi = Dataset(datapath+'rosie_UCL_SIT_ASSIM_CS2_AWI_SNAWI_ALPHA1_r243541_dcarneir.nc')

sits = [ctrl,base,nine,six,larm,cpom,smlg,awi,unc]
names = ['CTRL','BASE','\u03B1 = 0.9', '\u03B1 = 0.6','LARM','CPOM','SM-LG','AWI','UNC']

foam_lats = np.load('/Users/carmennab/Dropbox/met_office/data/auxiliary/lat_cent.npy')
foam_lons = np.load('/Users/carmennab/Dropbox/met_office/data/auxiliary/lon_cent.npy')

### season total

In [ ]:
variables = ['sidmassgrowthbot','sidmassgrowthwat','sidmasssi','sidmassmelttop','sidmassmeltbot',
            'sidmassmeltlat','sidmassth']
var_names = ['congelation','frazil growth','snow-ice growth','top melt','bottom melt',
         'lateral melt','total growth']

data = []
for sit, sit_name in zip(sits,names):
    print(sit_name)
    totals = {}
    for name, variable in zip(var_names,variables):
        area = np.array(sit['area'])
        rate = np.array(sit[variable])
        dens = icetype.copy()
        dens[np.isnan(dens)&~np.isnan(rate)] = rho_fyi # if icetype unknown, use fyi density
        mass = rate * area * 86400 # convert from rate per second to total mass per day
        vol = mass / dens
        vol_m = np.nansum(vol) # add up grid cells to get total pan-arctic volume
        totals[name] = vol_m * 0.0000000000001 # convert m^3 to 10^4 km^3 
    data.append(totals)     
    
df = pd.DataFrame(data,columns=var_names)
df

### season total relative to BASE

In [ ]:
totals = []
for c,d in enumerate(data):
    if c == 1: pass
    else:
        diffs = {}
        for var_name in var_names:
            diffs[var_name] = d[var_name] - data[1][var_name]
        totals.append(diffs)

df_base = pd.DataFrame(totals,columns=var_names)
df_base

### plot figure

In [ ]:
fig,axes = plt.subplots(figsize=(15,11.5),nrows=2, ncols=1)
plt.rc('font', size=15)
colors = ['#1f78b4','#ff7f00','#6a3d9a','#e31a1c','#33a02c','#fdbf6f','grey']

names = ['CTRL','BASE','\u03B1 = 0.9', '\u03B1 = 0.6','LARM','CPOM','SM-LG','AWI','UNC']
ax = df.plot.bar(ax=axes[0],color=colors)
ax.set_xticklabels(names,rotation=25)
ax.set_ylim([-3,4])
ax.set_yticks([-3,-2,-1,0,1,2,3,4])
ax.legend(var_names,bbox_to_anchor = (1, 1.025))
ax.set_title('total pan-Arctic growth and melt')
ax.set_ylabel('volume (10$^{4}$ km$^{3}$)')

names = ['CTRL','\u03B1 = 0.9', '\u03B1 = 0.6','LARM','CPOM','SM-LG','AWI','UNC']
ax = df_base.plot.bar(ax=axes[1],legend=False,color=colors)
ax.set_xticklabels(names,rotation=25)
ax.set_title('total pan-Arctic growth and melt relative to BASE ')
ax.set_ylabel('volume (10$^{4}$ km$^{3}$)')
ax.set_ylim([-1.5,0.5])
ax.set_yticks([-1.5,-1,-0.5,0,0.5])

fig.tight_layout()
datapath = '/Users/carmennab/Dropbox/Apps/Overleaf/FOAM sensitivity/main_figs/'
plt.savefig(datapath+'tmc_deconstructed.png',dpi=400, bbox_inches="tight")